<center><h1>Data Generation Experiments</h1></center>

<img src="logo_autodata.png" alt="drawing" width="200"/>

### Datasets

* Description of the datasets

### Metrics

* Mainly "adversarial scores" with different discriminant models.
* Do the different models agree on the discrepancy between distributions?
* Can we rely on this kind of metric?

### Generators

* Which generators seems to have the best guarantees of similarity and privacy?

In [1]:
import autopandas as apd
import numpy as np
from sklearn.neural_network import MLPClassifier
from IPython.display import display
%reload_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
def load_dataset(input_file, test_file=None):
    if test_file is None:
        data = apd.read_csv(input_file)
    else:
        train = apd.read_csv(input_file)
        test = apd.read_csv(test_file)
        data = apd.from_train_test(train, test)
    # processing
    data = data.imputation()
    data = data.encoding()
    data = data.normalization()
    # TMP reduce size
    #data = data[:min(len(data), 200)]
    return data

In [3]:
def describe_dataset(data):
    # TODO
    # read data
    pass
    # cat/num
    # descriptors
    # PCA, TSNE, LDA, y dist

In [4]:
def run_experiment(datasets_list, generators_list):
    # Generate data and compute metrics
    for dataset in datasets_list:
        data = load_dataset(dataset)
        print('\n')
        print(dataset)
        print('=================================================')
        describe_dataset(data)
        
        # to store results
        names = ['Generator']
        nnaas = ['NN AA']
        discrepancies = ['NN AA (sklearn)']
        discriminants1 = ['LogReg AA']
        discriminants2 = ['MLP AA']
        
        for generator in generators_list:
            # init generator
            if generator is None:
                generator = apd.generators.Copycat
            generator = generator()
            
            names.append(generator.__class__.__name__)
            
            # training
            generator.fit(data)
            gen_data = generator.sample(n=data.shape[0])
            
            # metrics
            nnaas.append(data.distance(gen_data, method='nnaa'))
            discrepancies.append(data.distance(gen_data))
            discriminants1.append(data.distance(gen_data, method='discriminant'))
            mlp = MLPClassifier()
            discriminants2.append(data.distance(gen_data, method='discriminant', model=mlp))
            
            # task score
            
            # plots
            # overlaying PCA and all
            #data.compare_marginals(gen_data, method='all')
            
        # Store results into a table
        results = np.array([names, nnaas, discrepancies, discriminants1, discriminants2]).T
        results = apd.AutoData(results[1:,:], columns=results[0,:])
        display(results)

In [5]:
path = 'autopandas/data/'
datasets = [path+x for x in ['iris.csv', 'wine.csv']] #, 'diabetes.csv', 'seeds.csv', 'adult.csv',
                             #'mushrooms.csv', 'wine.csv', 'squares.csv']]
# train/test boston, titanic
generators = [apd.generators.Copycat,
              apd.generators.KDE,
              apd.generators.ANM,
              apd.generators.Copula,
              apd.generators.GMM] # VAE, KDE, +

run_experiment(datasets, generators)



autopandas/data/iris.csv


/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,Generator,NN AA,NN AA (sklearn),LogReg AA,MLP AA
0,Copycat,0.0,0,0.5,0.5
1,KDE,0.57,0.6600000000000001,0.4888888888888889,0.9111111111111111
2,ANM,0.49,0.06000000000000005,0.5666666666666667,0.5666666666666667
3,Copula,0.6366666666666667,0.7066666666666666,0.45555555555555555,0.8333333333333334
4,GMM,0.64,0.5,0.5333333333333333,0.6222222222222222




autopandas/data/wine.csv


/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,Generator,NN AA,NN AA (sklearn),LogReg AA,MLP AA
0,Copycat,0.0,0,0.5,0.5
1,KDE,0.33166598611678233,0,0.4853641933287951,0.49625595643294756
2,ANM,0.5476725193956717,0.17905267456104523,0.4979577944179714,0.49727705922396187
3,Copula,0.6517966516945692,0.45304205798285,0.47345132743362833,0.8686181075561606
4,GMM,0.6536341363821968,0.5110249081257656,0.4863852961198094,0.7750170183798503
